 ## Project  : Amazon reviews analysis. This dataset consists of a few million Amazon customer reviews (input text) and star ratings (output labels) for learning how to train fastText for sentiment analysis.
dataset link:-  "https://www.kaggle.com/datasets/bittlingmayer/amazonreviews"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import bz2

In [ ]:
def get_labels_and_texts(file):
    labels = []
    texts = []
    for line in bz2.BZ2File(file):
        x = line.decode("utf-8")
        labels.append(int(x[9]) - 1)
        texts.append(x[10:].strip())
    labels = labels[:int(len(labels)*0.01)]
    texts = texts[:int(len(texts)*0.01)]
    return np.array(labels), texts
train_labels, train_texts = get_labels_and_texts('/content/drive/MyDrive/Colab Notebooks/DL assing 1N 2/train.ft.txt.bz2')
test_labels, test_texts = get_labels_and_texts('/content/drive/MyDrive/Colab Notebooks/DL assing 1N 2/test.ft.txt.bz2')

In [ ]:
train_df=pd.DataFrame(zip(train_texts,train_labels),columns=['text','label'])
print(train_df.head())
test_df=pd.DataFrame(zip(test_texts,test_labels),columns=['text','label'])
print(test_df.head())

                                                text  label
0  Stuning even for the non-gamer: This sound tra...      1
1  The best soundtrack ever to anything.: I'm rea...      1
2  Amazing!: This soundtrack is my favorite music...      1
3  Excellent Soundtrack: I truly like this soundt...      1
4  Remember, Pull Your Jaw Off The Floor After He...      1
                                                text  label
0  Great CD: My lovely Pat has one of the GREAT v...      1
1  One of the best game music soundtracks - for a...      1
2  Batteries died within a year ...: I bought thi...      0
3  works fine, but Maha Energy is better: Check o...      1
4  Great for the non-audiophile: Reviewed quite a...      1


In [ ]:
def remove_special_characters(text):
  text=text.str.lower()
  text=text.apply(lambda x: re.sub(r'[0-9]+','',x))
  text=text.apply(lambda x: re.sub(r'@mention',' ',x))
  text=text.apply(lambda x: re.sub(r'https?:\/\/\S+', ' ',x))
  text=text.apply(lambda x: re.sub(r"www.\[a-z]?\.?(com)+|[a-z]+\.(com)", ' ',x))
  text=text.apply(lambda x: re.sub(r"[_\,\>\(\-:\)\\\/\!\.\^\!\:\];='#]",'',x))
  return text

In [ ]:
train_df['text']=remove_special_characters(train_df['text'])
test_df['text']=remove_special_characters(test_df['text'])

In [ ]:
from keras.preprocessing import text,sequence

MAX_NB_WORDS = 10000
MAX_SEQUENCE_LENGTH = 250
EMBEDDING_DIM = 100
tokenizer = text.Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(train_df['text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 86207 unique tokens.


In [ ]:
!pip install keras_preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.6 MB/s eta 0:00:00


In [ ]:
from keras_preprocessing.sequence import pad_sequences

train_text = tokenizer.texts_to_sequences(train_df['text'].values)
train_text = pad_sequences(train_text, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', train_text.shape)

y = pd.get_dummies(train_df['label']).values
print('Shape of label tensor:', y.shape)

Shape of data tensor: (36000, 250)
Shape of label tensor: (36000, 2)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(train_text,y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(32400, 250) (32400, 2)
(3600, 250) (3600, 2)


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout,SpatialDropout1D,GlobalMaxPooling1D, Dense
import tensorflow as tf

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=train_text.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
# model.add(GlobalMaxPooling1D())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(2, activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 250, 100)          1000000   
                                                                 
 spatial_dropout1d (SpatialD  (None, 250, 100)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 128)               12928     
                                                                 
 dense_1 (Dense)             (None, 2)                 258       
                                                                 
Total params: 1,093,586
Trainable params: 1,093,586
Non-trainable params: 0
______________________________________________

In [ ]:
from keras.callbacks import EarlyStopping
epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)]) 

Epoch 1/5
456/456 [==============================] - 458s 988ms/step - loss: 0.3988 - accuracy: 0.8181 - val_loss: 0.2971 - val_accuracy: 0.8849
Epoch 2/5
456/456 [==============================] - 449s 986ms/step - loss: 0.2307 - accuracy: 0.9121 - val_loss: 0.2824 - val_accuracy: 0.8852
Epoch 3/5
456/456 [==============================] - 446s 978ms/step - loss: 0.1723 - accuracy: 0.9348 - val_loss: 0.2937 - val_accuracy: 0.8883
Epoch 4/5
456/456 [==============================] - 448s 980ms/step - loss: 0.1365 - accuracy: 0.9515 - val_loss: 0.3122 - val_accuracy: 0.8910
Epoch 5/5
456/456 [==============================] - 459s 1s/step - loss: 0.1099 - accuracy: 0.9611 - val_loss: 0.3387 - val_accuracy: 0.8935


In [ ]:
text = "I really enjoyed this movie. The acting was great and the plot was engaging. Highly recommend!"
# preprocess the text data
# text = preprocess_text(text)
text_sequence = tokenizer.texts_to_sequences([text])
padded_sequence = pad_sequences(text_sequence, maxlen=MAX_SEQUENCE_LENGTH)
prediction = model.predict(padded_sequence)
predicted_class = np.argmax(prediction)
sentiment = "positive" if predicted_class == 1 else "negative"
print("The sentiment of the text is:", sentiment)

1/1 [==============================] - 1s 1s/step
The sentiment of the text is: positive


In [ ]:
text = "The product was a complete waste of money. It broke within a week."

text_sequence = tokenizer.texts_to_sequences([text])
padded_sequence = pad_sequences(text_sequence, maxlen=MAX_SEQUENCE_LENGTH)
prediction = model.predict(padded_sequence)
predicted_class = np.argmax(prediction)
sentiment = "positive" if predicted_class == 1 else "negative"
print("The sentiment of the text is:", sentiment)

1/1 [==============================] - 0s 66ms/step
The sentiment of the text is: negative


In [ ]:
text = "I was really disappointed with this hotel."

text_sequence = tokenizer.texts_to_sequences([text])
padded_sequence = pad_sequences(text_sequence, maxlen=MAX_SEQUENCE_LENGTH)
prediction = model.predict(padded_sequence)
predicted_class = np.argmax(prediction)
sentiment = "positive" if predicted_class == 1 else "negative"
print("The sentiment of the text is:", sentiment)

1/1 [==============================] - 0s 62ms/step
The sentiment of the text is: negative


In [ ]:
text = "The room was clean and the staff was helpful."

text_sequence = tokenizer.texts_to_sequences([text])
padded_sequence = pad_sequences(text_sequence, maxlen=MAX_SEQUENCE_LENGTH)
prediction = model.predict(padded_sequence)
predicted_class = np.argmax(prediction)
sentiment = "positive" if predicted_class == 1 else "negative"
print("The sentiment of the text is:", sentiment)

1/1 [==============================] - 0s 125ms/step
The sentiment of the text is: positive
